In [57]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import KFold
import csv

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [58]:
train_df=pd.read_csv('C:/Users/Dell/OneDrive/Documents/Data mining assignment/DM assignment 1/1662145253_9967718_new_train.csv',header=None)
train_df.columns=['rating','review']
train_df

,rating,review
0,-1,"Eat at Fioris, they said. Youll like it, they..."
1,-1,I just don't understand the appeal. I've trie...
2,1,This is my go to place for a really good beef ...
3,-1,"Not impressed. When I ordered the Oyako bowl, ..."
4,-1,This is the first time ever I wrote a bad revi...
...,...,...
17995,-1,I was referred to go to this place by a buddy ...
17996,1,The food here was really good. We started off...
17997,1,I eat at this place maybe 2-3 a week! I am die...
17998,1,Phoenix Airport is getting better day by day. ...


In [59]:
test_df=pd.read_csv('C:/Users/Dell/Downloads/1662145254_0395114_new_test.csv',header=None)
test_df.columns=['review']

In [7]:
train_list=[]
sb=SnowballStemmer(language='english')
for i in range(len(train_df)):
    #using regular expressions to take words without any special characters
    value=re.sub('[^a-zA-Z]',' ',train_df.review[i])
    #replacing the new line character with  an empty string
    value=value.replace("\\n\\","")
    #spliting the line into words and taking them into list.
    words=value.split()
    #removing stopwords and then stemming the words to consider only important values from the list.
    words_nostopwords=[sb.stem(word) for word in words if not word in set(stopwords.words('english'))]
    words_lower=[word.lower() for word in words_nostopwords if len(word)>2]
    words_join=' '.join(words_lower)
    train_list.append(words_join) 
train_list

['eat fiori said youll like said nis conveni locat west liberti boulevard restaur featur absolut nowher park road imposs get nwhen call place order miser sound answer phone side said need place order said come least hour get cowork anyth right that realli unprofession some peopl may think charact realli care kind day person take pizza order want place order without feel bother nyou know illeg park car pick order place filthi insid pack full peopl total mayhem would never ever recommend eat store even place clean still peopl wait pick three order stand right also hundr degre they also take cash mention sign glad cash hand knew someth pizza start leak lid pizza drive home the bottom top rate greas how even happen friend said sign great pizza guy eat prioriti disgust you suppos take slice pizza pizza box top slide due thebgreas restaur could get away pizza greasi vincent gone could caulk bathtub chees rubberi disgust red sauc tast like ketchup sugar gross understand anybodi like restaur p

In [8]:
test_list=[]
sb=SnowballStemmer(language='english')
for i in range(len(test_df)):
    value=re.sub('[^a-zA-Z]',' ',test_df.review[i])
    value=value.replace("\\n\\","")
    words=value.split()
    words_nostopwords=[sb.stem(word) for word in words if not word in set(stopwords.words('english'))]
    words_lower=[word.lower() for word in words_nostopwords if len(word)>2]
    words_join=' '.join(words_lower)
    test_list.append(words_join) 
test_list

['got take last night horribl someth must happen peopl work veri disorgan should check food left order fajita littl meat beyond ridicul hard pepper plethora look like stew tomato look gringa know look right could way better job the rice miss complet one giant pizza pie look tortilla call person answer could bare speak english realli big deal book manag around resolv situat ask email could send note experi unhelp all offer come back right hungri fianc option get grumpi know better mess grumpi hungri pant',
 'girl sweet price reason the stand bed hot make sure adjust time might super white lol',
 'rudest peopl ever encount husband wife own busi call servic wife unbeliev unnecessarili rude demand don wast time call',
 'this airport covet destin lead flier view strip thing redeem hotel the tacki slot machin center termin one thing dont even smoker loung vega god sake there mani option food airport also tini airport lot traffic the traffic line doom catastrpoh defens drink line saw mani peo

In [9]:
train_rating=train_df['rating']

In [55]:
cv=CountVectorizer(max_features=2000)
train_transform=cv.fit_transform(train_list).toarray()
test_transform=cv.fit_transform(test_list).toarray()

In [40]:
folds=KFold(n_splits=5)
for train,test in folds.split(train_list):
    reviews_train, reviews_test = np.array(train_list)[train], np.array(train_list)[test]
    sentiments_train, sentiments_test = np.array(train_rating)[train], np.array(train_rating)[test]

In [ ]:
def KNN(k, test_transform, train_transform, test_list, train_rating):
    test_rating = []    
    for i in range(len(test_list)):
        #similarity between train and test values are calculated
        cosine_values = cosine_similarity(test_transform[i].reshape(1,-1), train_transform).T
        cosine_values =cosine_values.flatten()
        #using argsort method index values has been taken
        index = cosine_values.argsort()[:-k:-1]
        #using index values ratings are taken from train set and taken into list
        rating_list = train_rating[index].tolist()
        rating_list=[int(i) for i in rating_list]
        result = sum(rating_list)
        #test value is predicted based on the given condition
        test_rating.append("+1" if result>0 else "-1")  
    return test_rating
predicts = KNN(87,test_transform, train_transform, test_list, train_rating)  

In [54]:
f=open('final_result.csv','w')
for i in predicts:
    f.write(str(i)+"\n")
f.close()